In [26]:
import cartopy
import shapely
from shapely.geometry import MultiPolygon, Polygon, Point, LineString
import geopy
import itertools
import numpy as np
import pandas as pd
import xarray as xr
import os
import pickle
from pykml.factory import KML_ElementMaker as KML
import lxml
from tqdm import tqdm

In [27]:
read_folder = './shoreline_filter'
result_df_fn = 'result_df'

#Save datadir and resulting dataframe
with open(os.path.join(read_folder, result_df_fn),'rb') as f_r:
    (result_df, result_type_description) = pickle.load(f_r)

result_df

,file_name,result_type,data_before_landfilter,data_after_landfilter,proportion_close_to_shore,min_long,max_long,mean_long,min_lat,max_lat,mean_lat
0,GL_TS_MO_56014.nc,E,594,594,0.0,115.57,115.57,115.57,-32.450001,-32.450001,-32.450001
0,GL_TS_MO_45172.nc,C,0,0,None,None,None,None,None,None,None
0,GL_TS_MO_3301541.nc,A,None,None,None,None,None,None,None,None,None
0,GL_TS_MO_MEDS073.nc,C,0,0,None,None,None,None,None,None,None
0,GL_TS_MO_GoodrichBank01.nc,E,8046,8046,0.0,129.999924,130.001877,130.001068,-10.41755,-10.41552,-10.416465
...,...,...,...,...,...,...,...,...,...,...,...
0,GL_TS_MO_45144.nc,C,0,0,None,None,None,None,None,None,None
0,GL_TS_MO_55481.nc,E,1245,1245,0.0,154.449997,154.449997,154.449997,-27.65,-27.65,-27.649998
0,GL_TS_MO_46086.nc,E,234985,234985,0.0,-118.052002,-118.052002,-118.052002,32.499001,32.499001,32.499008
0,GL_TS_MO_44174.nc,C,0,0,None,None,None,None,None,None,None


In [28]:
for t, description in result_type_description.items():
    print(t, ':', description)
    display(result_df[result_df['result_type'] == t])

A : No valid parameters


,file_name,result_type,data_before_landfilter,data_after_landfilter,proportion_close_to_shore,min_long,max_long,mean_long,min_lat,max_lat,mean_lat
0,GL_TS_MO_3301541.nc,A,None,None,None,None,None,None,None,None,None
0,GL_WS_MO_45137.nc,A,None,None,None,None,None,None,None,None,None
0,GL_TS_MO_IF000619.nc,A,None,None,None,None,None,None,None,None,None
0,GL_TS_MO_7100061.nc,A,None,None,None,None,None,None,None,None,None
0,AR_TS_MO_AmundsenGulf-AT2.nc,A,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
0,GL_WS_MO_46146.nc,A,None,None,None,None,None,None,None,None,None
0,GL_WS_MO_46023.nc,A,None,None,None,None,None,None,None,None,None
0,GL_TS_MO_53960.nc,A,None,None,None,None,None,None,None,None,None
0,AR_TS_MO_BSO2B.nc,A,None,None,None,None,None,None,None,None,None


B : Exception during time filtering


,file_name,result_type,data_before_landfilter,data_after_landfilter,proportion_close_to_shore,min_long,max_long,mean_long,min_lat,max_lat,mean_lat
0,MO_TS_MO_ESTELLENCS.nc,B,None,None,None,None,None,None,None,None,None
0,MO_TS_MO_VIDA.nc,B,None,None,None,None,None,None,None,None,None
0,BS_TS_MO_VarnaBuoySURF.nc,B,None,None,None,None,None,None,None,None,None
0,MO_TS_MO_NADR-S1.nc,B,None,None,None,None,None,None,None,None,None
0,BS_TS_MO_BurgasBuoySURF.nc,B,None,None,None,None,None,None,None,None,None
0,MO_TS_MO_Molo-Bandiera.nc,B,None,None,None,None,None,None,None,None,None


C : No data for the time period


...

,file_name,result_type,data_before_landfilter,data_after_landfilter,proportion_close_to_shore,min_long,max_long,mean_long,min_lat,max_lat,mean_lat
0,GL_TS_MO_45195.nc,F,15164,0,1.0,-73.338997,-73.338997,-73.338997,44.487999,44.487999,44.487999
0,IR_TS_MO_Tenerife-coast-buoy.nc,F,17346,0,1.0,-16.24,-16.23,-16.23978,28.459999,28.459999,28.459993
0,NO_TS_MO_SchiermonnikoogWaggen.nc,F,14822,0,1.0,6.203462,6.203462,6.203461,53.4701,53.4701,53.470097
0,GL_TS_MO_45141.nc,F,3299,0,1.0,-115.315002,-115.315002,-115.31498,61.18,61.18,61.179996
0,GL_TS_MO_45029.nc,F,73206,0,1.0,-86.272003,-86.272003,-86.27198,42.900002,42.900002,42.900002
...,...,...,...,...,...,...,...,...,...,...,...
0,GL_TS_MO_45028.nc,F,72740,0,1.0,-91.829002,-91.829002,-91.828987,46.813999,46.813999,46.814003
0,GL_TS_MO_45187.nc,F,106295,0,1.0,-87.778999,-87.778999,-87.778999,42.491001,42.491001,42.491001
0,NO_TS_MO_NieuwpoortWind.nc,F,51223,0,1.0,2.716667,2.716667,2.716667,51.155556,51.155556,51.15556
0,GL_TS_MO_ljpc1.nc,F,17080,0,1.0,-117.257004,-117.257004,-117.256989,32.867001,32.867001,32.867001


G : Unique coordinates overlap


,file_name,result_type,data_before_landfilter,data_after_landfilter,proportion_close_to_shore,min_long,max_long,mean_long,min_lat,max_lat,mean_lat
0,GL_TS_MO_23015.nc,G,4176,4176,0.0,-149.177994,67.046997,66.182121,-36.778,0.475,0.174621
0,GL_TS_MO_55038.nc,G,30160,22598,0.250729,151.5,151.50238,151.501221,-23.9,-23.895611,-23.897774
0,GL_TS_MO_55037.nc,G,26340,26340,0.0,153.580002,153.949997,153.758026,-28.280001,-28.18,-28.228113
0,GL_TS_MO_55017.nc,G,5961,5961,0.0,153.470001,153.699997,153.679733,-29.120001,-28.870001,-28.870674
0,GL_TS_MO_55045.nc,G,27812,6188,0.777506,153.25,153.25,153.25,-27.4,-27.4,-27.399996
0,GL_TS_MO_55018.nc,G,5720,5720,0.0,153.229996,153.5,153.278259,-30.370001,-29.82,-30.367571
0,GL_TS_MO_55022.nc,G,3051,3051,0.0,151.020004,151.399994,151.020264,-34.48,-33.779999,-34.471169
0,GL_TS_MO_55046.nc,G,19682,19682,0.0,152.5,152.820007,152.640457,-25.1,-24.67,-24.858723
0,IR_TS_MO_1300130.nc,G,24780,24771,0.000363,-15.8087,-15.7959,-15.804513,28.187,28.2017,28.192307
0,IR_TS_MO_Melilla-coast-buoy.nc,G,16848,15156,0.100427,-2.944,-2.94,-2.940188,35.32,35.327,35.320335


H : Solved with bruteforce


,file_name,result_type,data_before_landfilter,data_after_landfilter,proportion_close_to_shore,min_long,max_long,mean_long,min_lat,max_lat,mean_lat


In [29]:
#Print enties with proportion_close_to_shore not in [0,1] and not nan
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(result_df[
        (~result_df['proportion_close_to_shore'].isin([0,1])) &
        (~result_df.isna().apply(any, axis=1)) 
        ].sort_values('proportion_close_to_shore'))

,file_name,result_type,data_before_landfilter,data_after_landfilter,proportion_close_to_shore,min_long,max_long,mean_long,min_lat,max_lat,mean_lat
0,IR_TS_MO_1300130.nc,G,24780,24771,0.000363,-15.8087,-15.7959,-15.804513,28.187,28.2017,28.192307
0,IR_TS_MO_Malaga-coast-buoy.nc,G,17542,17534,0.000456,-4.4444,-4.415,-4.440382,36.657101,36.692001,36.659531
0,IR_TS_MO_1300131.nc,G,20949,20874,0.00358,-16.6492,-16.5448,-16.600239,27.9834,28.0054,27.992958
0,IR_TS_MO_Melilla-coast-buoy.nc,G,16848,15156,0.100427,-2.944,-2.94,-2.940188,35.32,35.327,35.320335
0,GL_TS_MO_55033.nc,G,25522,22114,0.133532,151.070007,151.071335,151.070709,-23.305321,-23.299999,-23.302763
0,IR_TS_MO_LasPalmas-coast-buoy.nc,G,16512,13992,0.152616,-15.39,-15.39,-15.39,28.049999,28.049999,28.049997
0,GL_TS_MO_55038.nc,G,30160,22598,0.250729,151.5,151.50238,151.501221,-23.9,-23.895611,-23.897774
0,GL_TS_MO_55027.nc,G,28932,21612,0.253007,153.270004,153.279999,153.279388,-26.9,-26.8985,-26.899202
0,GL_TS_MO_52121.nc,G,25534,13480,0.472076,141.679993,141.679993,141.679977,-12.68,-12.68,-12.679999
0,GL_TS_MO_55036.nc,G,27312,6868,0.748535,153.729996,153.729996,153.729996,-28.6,-28.6,-28.600002


In [30]:
#Entierly ofshore
display(result_df[result_df['proportion_close_to_shore'] == 0])

,file_name,result_type,data_before_landfilter,data_after_landfilter,proportion_close_to_shore,min_long,max_long,mean_long,min_lat,max_lat,mean_lat
0,GL_TS_MO_56014.nc,E,594,594,0.0,115.57,115.57,115.57,-32.450001,-32.450001,-32.450001
0,GL_TS_MO_GoodrichBank01.nc,E,8046,8046,0.0,129.999924,130.001877,130.001068,-10.41755,-10.41552,-10.416465
0,GL_TS_MO_46001.nc,E,34345,34345,0.0,-147.949005,-147.919998,-147.948975,56.231998,56.304001,56.232044
0,NO_TS_MO_AWG.nc,E,49754,49754,0.0,5.95,5.95,5.95,53.5,53.5,53.5
0,BO_TS_MO_Koster.nc,E,355600,355600,0.0,11.0964,11.1005,11.097997,58.882099,58.8839,58.883144
...,...,...,...,...,...,...,...,...,...,...,...
0,AR_TS_MO_Kogurdufl.nc,E,8743,8743,0.0,-13.6265,-13.6265,-13.626499,65.64817,65.64817,65.648178
0,GL_TS_MO_51211.nc,E,30974,30974,0.0,-157.959,-157.959,-157.959,21.297001,21.297001,21.296999
0,GL_TS_MO_2300492.nc,E,17208,17208,0.0,72.193001,72.209999,72.209816,10.867,10.882,10.873935
0,GL_TS_MO_55481.nc,E,1245,1245,0.0,154.449997,154.449997,154.449997,-27.65,-27.65,-27.649998


In [31]:
#Entierly close to shore
display(result_df[result_df['proportion_close_to_shore'] == 1])

,file_name,result_type,data_before_landfilter,data_after_landfilter,proportion_close_to_shore,min_long,max_long,mean_long,min_lat,max_lat,mean_lat
0,GL_TS_MO_45195.nc,F,15164,0,1.0,-73.338997,-73.338997,-73.338997,44.487999,44.487999,44.487999
0,IR_TS_MO_Tenerife-coast-buoy.nc,F,17346,0,1.0,-16.24,-16.23,-16.23978,28.459999,28.459999,28.459993
0,NO_TS_MO_SchiermonnikoogWaggen.nc,F,14822,0,1.0,6.203462,6.203462,6.203461,53.4701,53.4701,53.470097
0,GL_TS_MO_45141.nc,F,3299,0,1.0,-115.315002,-115.315002,-115.31498,61.18,61.18,61.179996
0,GL_TS_MO_45029.nc,F,73206,0,1.0,-86.272003,-86.272003,-86.27198,42.900002,42.900002,42.900002
...,...,...,...,...,...,...,...,...,...,...,...
0,GL_TS_MO_45028.nc,F,72740,0,1.0,-91.829002,-91.829002,-91.828987,46.813999,46.813999,46.814003
0,GL_TS_MO_45187.nc,F,106295,0,1.0,-87.778999,-87.778999,-87.778999,42.491001,42.491001,42.491001
0,NO_TS_MO_NieuwpoortWind.nc,F,51223,0,1.0,2.716667,2.716667,2.716667,51.155556,51.155556,51.15556
0,GL_TS_MO_ljpc1.nc,F,17080,0,1.0,-117.257004,-117.257004,-117.256989,32.867001,32.867001,32.867001
